# Getting recommendations using Amazon Personalization and AWS Data Exchange

Recommended Time: 90 Min

## Prerequisites

To use this notebook you need to be suscribed to 

### Import Dependencies and Setup Boto3 Python Clients

Throughout this workshop we will need access to some common libraries and clients for connecting to AWS services. We also have to retrieve Uid from a SageMaker notebook instance tag.

In [ ]:
# Import Dependencies

import boto3
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import requests
import csv
import sys
import botocore
import uuid

from datetime import datetime
from datetime import date
from packaging import version
from random import randint
from botocore.exceptions import ClientError


%matplotlib inline

# Setup Clients

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client('personalize-events')
s3 = boto3.client('s3')

with open('/opt/ml/metadata/resource-metadata.json') as f:
  data = json.load(f)
sagemaker = boto3.client('sagemaker')
sagemakerResponce = sagemaker.list_tags(ResourceArn=data["ResourceArn"])
for tag in sagemakerResponce["Tags"]:
    if tag['Key'] == 'Uid':
        Uid = tag['Value']
        break

### Implement some visualization functions for displaying information of the products in a dataframe

Throughout this workshop we will need to search information of products several times, this function will help us to do it without repeating the same code.

In [ ]:
def search_items_in_dataframe(item_list):
    df = pd.DataFrame() 
    for x in range(len(item_list)):
        temp = products_dataset_df.loc[products_dataset_df['ITEM_ID'] == int(item_list[x]['itemId'])]
        df = df.append(temp, ignore_index=True)
    pd.set_option('display.max_rows', 10)
    return df


### Configure Bucket and Data Output Location

We will be configuring some variables that will store the location of our source data. Substitute the name of the bucket we will create later with your own. 

In [ ]:

bucket = " "     # Use your own bucket
items_filename = "items.csv"                # Do Not Change
users_filename = "users.csv"                # Do Not Change
interactions_filename = "interactions.csv"  # Do Not Change

## Get, Prepare, and Upload User, Product, and Interaction Data

First we need to create a bucket to store the datasets for Personalize to consume them. 

Download datasets.

Let's get started.

In [ ]:
# Substitute the '/dataset/object-key' by the key of the file copied by ADX containing the weather dataset

s3.download_file(bucket, '/dataset/object-key', 'weather_data.csv')

#### Download and Explore and clean the Weather Dataset

In [ ]:
weather_df = pd.read_csv('weather_data.csv')
pd.set_option('display.max_rows', 5)
weather_df

In [ ]:
#Create a new DF with all the values related to Atlanta 
weather_df= weather_df[weather_df.city == 'Atlanta']
weather_df

In [ ]:
#Helper function to find the temperature in weather dataset

def find_weather_data_by_timestamp(timestamp):
    date_ds = str(date.fromtimestamp(timestamp)).replace('-', '')
    data = weather_df.loc[weather_df['date'] == int(date_ds)]
    daily_temp = float (data['avgTemp'])
    daily_temp = int ((daily_temp - 32) * 5.0/9.0)
    return daily_temp


In [ ]:
#test a sample timestamp
find_weather_data_by_timestamp(1587846700)

#### Download and Explore and clean the Products Dataset

In [ ]:
products_df = pd.read_csv('./items-origin.csv')
pd.set_option('display.max_rows', 5)
products_df


### Clean the product dataset and drop columms we don't need.


#### Prepare products Data

When training models in Amazon Personalize, we can provide meta data about our items. For this workshop we will add each product's category and style to the item dataset. The product's unique identifier is required. Then we will rename the columns in our dataset to match our schema (defined later) and those expected by Personalize. Finally, we will save our dataset as a CSV and copy it to our S3 bucket.

In [ ]:
products_dataset_df = products_df[['id','category','type', 'size']]


In [ ]:
products_dataset_df['category'].unique()

In [ ]:
products_dataset_df = products_dataset_df.rename(columns = {'id':'ITEM_ID','category':'CATEGORY','type':'TYPE', 'size':'SIZE'}) 


In [ ]:
pd.set_option('display.max_rows', 5)
products_dataset_df

In [ ]:
products_dataset_df.to_csv(items_filename, index=False)

#### Download and Explore the Users Dataset

In [ ]:
users_df = pd.read_csv('./users-origin.csv')
pd.set_option('display.max_rows', 5)
users_df

#### Prepare products Data

Similar to the items dataset we created above, we can provide metadata on our users when training models in Amazon Personalize. For this workshop we will include each user's id and persona. As before, we will name the columns to match our schema, save the data as a CSV, and upload to our S3 bucket.

In [ ]:
users_dataset_df = users_df[['id','persona']]
users_dataset_df = users_dataset_df.rename(columns = {'id':'USER_ID','persona':'PERSONA'}) 
users_dataset_df.head(5)

users_dataset_df.to_csv(users_filename, index=False)

In [ ]:
users_dataset_df.head(5)

In [ ]:
products_dataset_df.info()

### Create User-Items Interactions Dataset

To mimic user behavior, we will be generating a new dataset that represents user interactions with items. To make the interactions more realistic, we will use a predefined shopper persona for each user to generate event types for products matching that persona. This persona is composed by 3 categories, separated by the symbol "_". 
The upsampling process will create events for viewing products, add products to a cart, checking out, and completing orders.

In [ ]:
%%time

# Minimum number of interactions to generate
min_interactions = 1000000

# Percentages of each event type to generate
product_added_percent = .08
cart_viewed_percent = .05
checkout_started_percent = .02
order_completed_percent = .01

# Count of interactions generated for each event type
product_viewed_count = 0
product_added_count = 0
cart_viewed_count = 0
checkout_started_count = 0
order_completed_count = 0

# How many days in the past (from initial date) to start generating interactions
days_back = 365

#selecting a start time between 2020/02/23 and 2020/10/22 to match the weather data from the sample
date_time_obj = datetime.strptime('2019-06-25 09:27:53', '%Y-%m-%d %H:%M:%S')
start_time = int(datetime.timestamp(date_time_obj))
#start_time = int(time.time())


next_timestamp = start_time - (days_back * 24 * 60 * 60)
seconds_increment = int((start_time - next_timestamp) / min_interactions)
next_update = start_time + 60

assert seconds_increment > 0, "Increase days_back or reduce min_interactions"

print('Minimum interactions to generate: {}'.format(min_interactions))
print('Days back: {}'.format(days_back))
print('Starting timestamp: {} ({})'.format(next_timestamp, time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(next_timestamp))))
print('Seconds increment: {}'.format(seconds_increment))

print("Generating interactions... (this may take a few minutes)")
interactions = 0

subsets_cache = {}

with open(interactions_filename, 'w') as outfile:
    f = csv.writer(outfile)
    f.writerow(["ITEM_ID", "USER_ID", "EVENT_TYPE", "TIMESTAMP", "DAILY_TEMPERATURE"])

    while interactions < min_interactions:
        #if (time.time() > next_update):
         #   rate = interactions / (time.time() - start_time)
          #  to_go = (min_interactions - interactions) / rate
           # print('Generated {} interactions so far ({:0.2f} seconds to go)'.format(interactions, to_go))
            #next_update += 60

        # Pick a random user
        user = users_df.sample().iloc[0]

        # Determine category affinity from user's persona
        persona = user['persona']
        preferred_categories = persona.split('_')       

        # Select category based on weighted preference of category order.
        category = np.random.choice(preferred_categories, 1, p=[0.6, 0.25, 0.15])[0]
        gender = user['gender']

        # Check if subset data frame is already cached for category & gender
        prods_subset_df = subsets_cache.get(category + gender)
                
        if prods_subset_df is None:
            # Select products from selected category without gender affinity or that match user's gender
            prods_subset_df = products_df.loc[(products_df['category'] == category)]
            # Update cache
            subsets_cache[category + gender] = prods_subset_df

        # Pick a random product from gender filtered subset
        product = prods_subset_df.sample().iloc[0]

        this_timestamp = next_timestamp + randint(0, seconds_increment)
        daily_temp = find_weather_data_by_timestamp(this_timestamp)
        f.writerow([product['id'],
                    user['id'], 
                    'ProductViewed',
                    this_timestamp,
                    daily_temp])

        next_timestamp += seconds_increment
        product_viewed_count += 1
        interactions += 1

        if product_added_count < int(product_viewed_count * product_added_percent):
            this_timestamp += randint(0, int(seconds_increment / 2))
            daily_temp = find_weather_data_by_timestamp(this_timestamp)
            f.writerow([product['id'],
                        user['id'], 
                        'ProductAdded',
                        this_timestamp,
                        daily_temp])
            interactions += 1
            product_added_count += 1

        if cart_viewed_count < int(product_viewed_count * cart_viewed_percent):
            this_timestamp += randint(0, int(seconds_increment / 2))
            daily_temp = find_weather_data_by_timestamp(this_timestamp)
            f.writerow([product['id'],
                        user['id'], 
                        'CartViewed',
                        this_timestamp,
                        daily_temp])
            interactions += 1
            cart_viewed_count += 1

        if checkout_started_count < int(product_viewed_count * checkout_started_percent):
            this_timestamp += randint(0, int(seconds_increment / 2))
            daily_temp = find_weather_data_by_timestamp(this_timestamp)
            f.writerow([product['id'],
                        user['id'], 
                        'CheckoutStarted',
                        this_timestamp,
                        daily_temp])
            interactions += 1
            checkout_started_count += 1

        if order_completed_count < int(product_viewed_count * order_completed_percent):
            this_timestamp += randint(0, int(seconds_increment / 2))
            daily_temp = find_weather_data_by_timestamp(this_timestamp)
            f.writerow([product['id'],
                        user['id'], 
                        'OrderCompleted',
                        this_timestamp,
                        daily_temp])
            interactions += 1
            order_completed_count += 1
    
print("Done")
print("Total interactions: " + str(interactions))
print("Total product viewed: " + str(product_viewed_count))
print("Total product added: " + str(product_added_count))
print("Total cart viewed: " + str(cart_viewed_count))
print("Total checkout started: " + str(checkout_started_count))
print("Total order completed: " + str(order_completed_count))

#### Open and Explore the Interactions Dataset

In [ ]:
interactions_df = pd.read_csv(interactions_filename)
interactions_df

Chart the counts of each `EVENT_TYPE` generated for the interactions dataset. We're simulating a site where visitors heavily view/browse products and to a lesser degree add products to their cart and checkout.

In [ ]:
categorical_attributes = interactions_df.select_dtypes(include = ['object'])

plt.figure(figsize=(16,3))
sns.countplot(data = categorical_attributes, x = 'EVENT_TYPE')

Take note of the DAILY_TEMPERATURE values included in our interaction dataset, you will be using them for getting recomendations. 

In [ ]:
interactions_df['DAILY_TEMPERATURE'].unique()

#### Upload Data
Now we will upload the data we prepared to S3.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(interactions_filename).upload_file(interactions_filename)

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(items_filename).upload_file(items_filename)

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(users_filename).upload_file(users_filename)

## Configure Amazon Personalize

Now that we've prepared our three datasets and uploaded them to S3 we'll need to configure the Amazon Personalize service to understand our data so that it can be used to train models for generating recommendations.

### Create Schemas for Datasets

Amazon Personalize requires a schema for each dataset so it can map the columns in our CSVs to fields for model training. Each schema is declared in JSON using the [Apache Avro](https://avro.apache.org/) format.

Let's define and create schemas in Personalize for our datasets.

#### Items Datsaset Schema

In [ ]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "CATEGORY",
            "type": "string",
            "categorical": True,
        },
        {
            "name": "TYPE",
            "type": "string",
            "categorical": True,
        },
        {
            "name": "SIZE",
            "type": "string",
            "categorical": True,
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "adx-weather-schema-items",
    schema = json.dumps(items_schema)
)

items_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

#### Users Dataset Schema

In [ ]:
users_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "PERSONA",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "adx-weather-users",
    schema = json.dumps(users_schema)
)

users_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

#### Interactions Dataset Schema

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "DAILY_TEMPERATURE",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "adx-weather-interactions",
    schema = json.dumps(interactions_schema)
)

interactions_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### Create and Wait for Dataset Group

Next we need to create the dataset group that will contain our three datasets.

#### Create Dataset Group

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = 'adx-weather-dataset'
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

print(f'DatasetGroupArn = {dataset_group_arn}')

#### Wait for Dataset Group to Have ACTIVE Status

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### Create Items Dataset

Next we will create the datasets in Personalize for our three dataset types. Let's start with the items dataset.

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "adx-weather-dataset-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Create Users Dataset

In [ ]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    name = "adx-weather-dataset-users",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Create Interactions Dataset

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "adx-weather-dataset-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Import Datasets to Personalize

Up to this point we have generated CSVs containing data for our users, items, and interactions and staged them in an S3 bucket. We also created schemas in Personalize that define the columns in our CSVs. Then we created a datset group and three datasets in Personalize that will receive our data. In the following steps we will create import jobs with Personalize that will import the datasets from our S3 bucket into the service.

### Setup Permissions

By default, the Personalize service does not have permission to acccess the data we uploaded into the S3 bucket in our account. In order to grant access to the  Personalize service to read our CSVs, we need use the role we created in the pre-requisites steps. 

In [ ]:
# Put your role arn here. 
role_arn = ""

### Create Import Jobs

With the permissions in place to allow Personalize to access our CSV files, let's create three import jobs to import each file into its respective dataset. Each import job can take several minutes to complete so we'll create all three and then wait for them all to complete.

#### Create Items Dataset Import Job

In [ ]:
items_create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "adx-weather-dataset-items-import-job",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, items_filename)
    },
    roleArn = role_arn
)

items_dataset_import_job_arn = items_create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(items_create_dataset_import_job_response, indent=2))

#### Create Users Dataset Import Job

In [ ]:
users_create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "adx-weather-dataset-users-import-job",
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, users_filename)
    },
    roleArn = role_arn
)

users_dataset_import_job_arn = users_create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(users_create_dataset_import_job_response, indent=2))

#### Create Interactions Dataset Import Job

In [ ]:
interactions_create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "adx-weather-dataset-interactions-import-job",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, interactions_filename)
    },
    roleArn = role_arn
)

interactions_dataset_import_job_arn = interactions_create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(interactions_create_dataset_import_job_response, indent=2))

### Wait for Import Jobs to Complete

It will take 10-15 minutes for the import jobs to complete, while you're waiting you can learn more about Datasets and Schemas here: https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html

We will wait for all three jobs to finish.

#### Wait for Items Import Job to Complete

In [ ]:
%%time

import_job_arns = [ items_dataset_import_job_arn, users_dataset_import_job_arn, interactions_dataset_import_job_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for job_arn in reversed(import_job_arns):
        import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = job_arn
        )
        status = import_job_response["datasetImportJob"]['status']

        if status == "ACTIVE":
            print(f'Import job {job_arn} successfully completed')
            import_job_arns.remove(job_arn)
        elif status == "CREATE FAILED":
            print(f'Import job {job_arn} failed')
            if import_job_response.get('failureReason'):
                print('   Reason: ' + import_job_response['failureReason'])
            import_job_arns.remove(job_arn)

    if len(import_job_arns) > 0:
        print('At least one dataset import job still in progress')
        time.sleep(60)
    else:
        print("All import jobs have ended")
        break

## Create Solutions

With our three datasets imported into our dataset group, we can now turn to training models. 
When creating a solution, you provide your dataset group and the recipe for training. Let's declare the recipes that we will need for our solutions.

### List Recipes

First, let's list all available recipes.

In [ ]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

As you can see above, there are several recipes to choose from. Let's use only the Product Recommendations

#### Declare Personalize Recipe for Product Recommendations


In [ ]:
recommend_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

### Create Solutions and Solution Versions

With our recipes defined, we can now create our solutions and solution versions.

#### Create Product Recommendation Solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "adx-weather-product-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recommend_recipe_arn
)

recommend_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Create Product Recommendation Solution Version

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = recommend_solution_arn
)

recommend_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

### Wait for Solution Versions to Complete

It can take 40-60 minutes for all solution versions to be created. During this process a model is being trained and tested with the data contained within your datasets. The duration of training jobs can increase based on the size of the dataset, training parameters and using AutoML vs. manually selecting a recipe. We submitted requests for all three solutions and versions at once so they are trained in parallel and then below we will wait for all three to finish.

While you are waiting for this process to complete you can learn more about solutions here: https://docs.aws.amazon.com/personalize/latest/dg/training-deploying-solutions.html

#### Wait for Related Products Solution Version to Have ACTIVE Status

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    soln_ver_response = personalize.describe_solution_version(
        solutionVersionArn = recommend_solution_version_arn
    )
    status = soln_ver_response["solutionVersion"]["status"]
    time.sleep(10)
    print(status)
    if status == "ACTIVE":
        print(f'Solution version {recommend_solution_version_arn} successfully completed')
        break
    elif status == "CREATE FAILED":
        print(f'Solution version {soln_ver_arn} failed')
        if soln_ver_response.get('failureReason'):
            print('   Reason: ' + soln_ver_response['failureReason'])


### Evaluate Offline Metrics for Solution Versions

Amazon Personalize provides [offline metrics](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html#working-with-training-metrics-metrics) that allow you to evaluate the performance of the solution version before you deploy the model in your application. Metrics can also be used to view the effects of modifying a Solution's hyperparameters or to compare the metrics between solutions that use the same training data but created with different recipes.

Let's retrieve the metrics for the solution versions we just created.

#### Product Recommendations Metrics

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = recommend_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

## Create Campaigns

Once we're satisfied with our solution versions, we need to create Campaigns for each solution version. When creating a campaign you specify the minimum transactions per second (`minProvisionedTPS`) that you expect to make against the service for this campaign. Personalize will automatically scale the inference endpoint up and down for the campaign to match demand but will never scale below `minProvisionedTPS`.

Let's create campaigns for our three solution versions with each set at `minProvisionedTPS` of 1.

#### Create Product Recommendation Campaign

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "adx-weather-product-personalization",
    solutionVersionArn = recommend_solution_version_arn,
    minProvisionedTPS = 1
)

recommend_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Wait for Related Products Campaign to Have ACTIVE Status

It can take 20-30 minutes for the campaigns to be fully created. 

While you are waiting for this to complete you can learn more about campaigns here: https://docs.aws.amazon.com/personalize/latest/dg/campaigns.html

In [ ]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    campaign_response = personalize.describe_campaign(
        campaignArn = recommend_campaign_arn
    )
    status = campaign_response["campaign"]["status"]
    time.sleep(10)
    print(status)

    if status == "ACTIVE":
        print(f'Campaign {recommend_campaign_arn} successfully completed')
        break
    elif status == "CREATE FAILED":
        print(f'Campaign {campaign_arn} failed')
        if campaign_response.get('failureReason'):
            print('   Reason: ' + campaign_response['failureReason'])

## Test Campaigns

Now that our campaigns have been fully created, let's test each campaign and evaluate the results.

### Test Product Recommendations Campaign

Let's test the recommendations made by the product recommendations campaign by selecting a user from the users dataset and requesting item recommendations for that user.

#### Select a User

We'll just pick a random user for simplicity. Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [ ]:
# User with interactions 170
# Cold start user 7000
user_id = 160
users_df.loc[users_df['id'] == user_id]

**Take note of the `persona` value for the user above. We should see recommendations for products consistent with this persona since we generated historical interactions for products in the categories represented in the persona.**

#### Get Product Recommendations for User

Now let's call Amazon Personalize to get recommendations for our user from the product recommendations campaign.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = recommend_campaign_arn,
    userId = str(user_id),
    numResults = 5)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=4))
search_items_in_dataframe(item_list)

Are the recommended products consistent with the persona? Note that this is a rather contrived example using a limited amount of generated interaction data without model parameter tuning. The purpose is to give you hands on experience building models and retrieving inferences from Amazon Personalize. 

## Contextual recomendations

Now lets explore the possibility of passing contextual information to the recomendation call. Context can be any attribute included in the Interactions dataset used to train the solution. in our case we included the average temperature of each day in Atlanta extracted from WeatherTrends360 dataset:https://aws.amazon.com/marketplace/pp/prodview-wbuuretid73x4?sr=0-3&ref_=beagle&applicationId=AWSMPContessa#offers.

If you want to try your own solutions feel free to explore other dataset on the AWS Marketplace.  

Other useful contextual informacion can be the device or trade channel used to interact and other similar metadata alike. More information: https://aws.amazon.com/blogs/machine-learning/increasing-the-relevance-of-your-amazon-personalize-recommendations-by-leveraging-contextual-information/

Lets select a user and test the recomendations for the included temperature ranges.


In [ ]:
user_id = 7000
users_dataset_df.loc[users_dataset_df['USER_ID'] == user_id]


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = recommend_campaign_arn,
    userId = str(user_id),
    numResults = 5,
    context = {
      'DAILY_TEMPERATURE': '30'
    })

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=4))
search_items_in_dataframe(item_list)

The items recommended are different from the previous calls? Try different users, values and amount of items recommended to get a grasp of the behavior. 

## Workshop Complete

Congratulations! You have completed the contextual Weather Personalization Workshop.

### Cleanup

You MUST run the cleanup notebook or manually clean up these resources. If using the notebook, save the names of the elements to be cleaned:

In [ ]:
%store dataset_group_arn
%store items_dataset_arn
%store users_dataset_arn
%store interactions_dataset_arn
%store role_arn
%store users_dataset_import_job_arn
%store interactions_dataset_import_job_arn
%store items_dataset_import_job_arn
%store recommend_campaign_arn